In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv(r'/kaggle/input/playground-series-s3e22/train.csv')
data1=pd.read_csv(r'/kaggle/input/playground-series-s3e22/test.csv')
data2=pd.read_csv(r'/kaggle/input/playground-series-s3e22/sample_submission.csv')

In [ ]:
data.head()

In [ ]:
data.head()

In [ ]:
data1.head()

In [ ]:
data2.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.shape

**Dealing With Missing Values**


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15,10))
plt.title("Visualization Of NULL Values")
sns.heatmap(data.isnull(),cmap='PiYG')

In [ ]:
data.dtypes

In [ ]:
data['temp_of_extremities']=data['temp_of_extremities'].fillna(data['temp_of_extremities'].mode()[0])
data['peripheral_pulse']=data['peripheral_pulse'].fillna(data['peripheral_pulse'].mode()[0])
data['mucous_membrane']=data['mucous_membrane'].fillna(data['mucous_membrane'].mode()[0])
data['capillary_refill_time']=data['capillary_refill_time'].fillna(data['capillary_refill_time'].mode()[0])
data['pain']=data['pain'].fillna(data['pain'].mode()[0])
data['rectal_exam_feces']=data['rectal_exam_feces'].fillna(data['rectal_exam_feces'].mode()[0])
data['abdomen']=data['abdomen'].fillna(data['abdomen'].mode()[0])
data['abdomo_appearance']=data['abdomo_appearance'].fillna(data['abdomo_appearance'].mode()[0])


In [ ]:
data=data.dropna()

In [ ]:
plt.figure(figsize=(15,10))
plt.title("Visualization Of NULL Values After Handling All Missing Values")
sns.heatmap(data.isnull(),cmap='PiYG')

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['surgery']=le.fit_transform(data['surgery'])
data['surgical_lesion']=le.fit_transform(data['surgical_lesion'])
data['cp_data']=le.fit_transform(data['cp_data'])
data['age']=le.fit_transform(data['age'])
data.head()

In [ ]:
data.head()

In [ ]:
unique_outcomes_values={}
for key,value in enumerate(data['outcome'].unique(),0):
    unique_outcomes_values[value]=key

In [ ]:
data['outcome']=data['outcome'].map(unique_outcomes_values)

In [ ]:
data.head()

In [ ]:
def map_unique_values_to_int(df, column_name):
    unique_values = df[column_name].unique()
    unique_values_mapping = {value: key for key, value in enumerate(unique_values, 0)}
    df[column_name] = df[column_name].map(unique_values_mapping)
map_unique_values_to_int(data, 'mucous_membrane')
map_unique_values_to_int(data, 'peripheral_pulse')
map_unique_values_to_int(data, 'abdomo_appearance')
map_unique_values_to_int(data, 'pain')
map_unique_values_to_int(data, 'capillary_refill_time')
map_unique_values_to_int(data, 'peristalsis')
map_unique_values_to_int(data, 'abdominal_distention')
map_unique_values_to_int(data, 'nasogastric_tube')
map_unique_values_to_int(data, 'nasogastric_reflux')
map_unique_values_to_int(data, 'rectal_exam_feces')
map_unique_values_to_int(data, 'abdomen')
map_unique_values_to_int(data, 'temp_of_extremities')

In [ ]:
data.dtypes

In [ ]:
data=data.drop(columns=['hospital_number','id'])

In [ ]:
x=data.iloc[:,:-1].values
x

In [ ]:
y=data.iloc[:,-1].values
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [ ]:
print("Shape Of X Training Dataset",x_train.shape)
print("Shape Of X Testing Dataset",x_test.shape)
print("Shape Of Y Training Dataset",y_train.shape)
print("Shape Of Y Testing Dataset",y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

**SVC MODEL**


In [ ]:
models=[
        ("SVM",SVC()),
        ("LogisticRegression",LogisticRegression()),
        ("DecisionTreeClassifier",DecisionTreeClassifier(random_state=0)),
        ("RandomForestClassifier",RandomForestClassifier(n_estimators=100)),
        ("KNeighborsClassifier",KNeighborsClassifier(n_neighbors=5,p=2,metric='minkowski')),
        ("NaiveBayes",GaussianNB())
        ]

In [ ]:
result = {}
for name, model in models:
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    report = classification_report(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    score = accuracy_score(y_test, y_pred) * 100

    print("============================================= CLASSIFICATION REPORT ===============================================")
    print(f"Model Name {name}")
    print(report)
    print("============================================= ACCURACY SCORE ===============================================")
    print(f"Model Name {name}")
    print(f"{name} Accuracy Score ::", score)
    result[name] = score

    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"Confusion Matrix for {name}")
    plt.show()

In [ ]:
result

In [ ]:
data1.head()

In [ ]:
data1['temp_of_extremities']=data1['temp_of_extremities'].fillna(data1['temp_of_extremities'].mode()[0])
data1['peripheral_pulse']=data1['peripheral_pulse'].fillna(data1['peripheral_pulse'].mode()[0])
data1['mucous_membrane']=data1['mucous_membrane'].fillna(data1['mucous_membrane'].mode()[0])
data1['capillary_refill_time']=data1['capillary_refill_time'].fillna(data1['capillary_refill_time'].mode()[0])
data1['pain']=data1['pain'].fillna(data1['pain'].mode()[0])
data1['rectal_exam_feces']=data1['rectal_exam_feces'].fillna(data1['rectal_exam_feces'].mode()[0])
data1['abdomen']=data1['abdomen'].fillna(data1['abdomen'].mode()[0])
data1['abdomo_appearance']=data1['abdomo_appearance'].fillna(data1['abdomo_appearance'].mode()[0])


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data1['surgery']=le.fit_transform(data1['surgery'])
data1['surgical_lesion']=le.fit_transform(data1['surgical_lesion'])
data1['cp_data']=le.fit_transform(data1['cp_data'])
data1['age']=le.fit_transform(data1['age'])
data1.head()

In [ ]:
def map_unique_values_to_int(df, column_name):
    unique_values = df[column_name].unique()
    unique_values_mapping = {value: key for key, value in enumerate(unique_values, 0)}
    df[column_name] = df[column_name].map(unique_values_mapping)
map_unique_values_to_int(data1, 'mucous_membrane')
map_unique_values_to_int(data1, 'peripheral_pulse')
map_unique_values_to_int(data1, 'abdomo_appearance')
map_unique_values_to_int(data1, 'pain')
map_unique_values_to_int(data1, 'capillary_refill_time')
map_unique_values_to_int(data1, 'peristalsis')
map_unique_values_to_int(data1, 'abdominal_distention')
map_unique_values_to_int(data1, 'nasogastric_tube')
map_unique_values_to_int(data1, 'nasogastric_reflux')
map_unique_values_to_int(data1, 'rectal_exam_feces')
map_unique_values_to_int(data1, 'abdomen')
map_unique_values_to_int(data1, 'temp_of_extremities')

data1.head()

In [ ]:
a=data1.drop(columns=['id','hospital_number'])
from sklearn.ensemble import RandomForestClassifier


In [ ]:
model=RandomForestClassifier(n_estimators=100)

In [ ]:
model.fit(x_train,y_train)

In [ ]:
y_pred=model.predict(a)

In [ ]:
y_pred

In [ ]:
data1['outcome']=y_pred

In [ ]:
data1=data1[['id','outcome']]
map_dict={0:'died', 1:'euthanized', 2:'lived'}
data1['outcome']=data1['outcome'].map(map_dict)
data1.head()